In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import math

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
class Model(object):
    def _batch_normal(self,x,training,epsilon=1e-3, decay=0.99):
        size = x.shape.as_list()[1]
        scale = tf.Variable(tf.ones((size,))*0.1,trainable=True)
        offset = tf.Variable(tf.zeros((size,)),trainable=True)

        pop_mean =  tf.Variable(tf.zeros((size,)),trainable=False)
        pop_var =  tf.Variable(tf.ones((size,)),trainable=False)
        batch_mean, batch_var = tf.nn.moments(x, [0])
        train_mean_op = tf.assign(pop_mean, pop_mean*decay+batch_mean*(1-decay))
        train_var_op = tf.assign(pop_var, pop_var*decay + batch_var*(1-decay))

        def batch_statistics():
            with tf.control_dependencies([train_mean_op, train_var_op]):
                return tf.nn.batch_normalization(x, batch_mean, batch_var, offset, scale, epsilon)

        def population_statistics():
            return tf.nn.batch_normalization(x, pop_mean, pop_var, offset, scale, epsilon)

        return tf.cond(training, batch_statistics, population_statistics),pop_mean,pop_var
    
    def __init__(self,l1_batch_normal,l2_batch_normal,x,y,drop_rate,isTrain,name:str):
        with tf.name_scope("Model_"+name.replace(' ','_')):
            self.name = name
            self.learning_rate = tf.Variable(0.001,trainable=False,name="learning_rate")
            epsilon = 1e-8
            _summary_merge_list = []
            with tf.name_scope("layer1"+("_BN" if l1_batch_normal else "")):
                #784-400-150-10神经网络(BN)
                wL1 = tf.Variable(tf.truncated_normal([784,400]))
                bL1 = tf.Variable(tf.zeros([400]))
                mL1 = tf.matmul(x,wL1) + bL1
                if l1_batch_normal:
                    batch_normal1,pop_mean,pop_var = self._batch_normal(mL1,isTrain,epsilon)
                    _summary_merge_list.append(tf.summary.histogram("Mean",pop_mean))
                    _summary_merge_list.append(tf.summary.histogram("Var",pop_var))
                    rL1 = tf.nn.sigmoid(batch_normal1) 
                else:
                    rL1 = tf.nn.sigmoid(mL1) 
                rL1 = tf.nn.dropout(rL1,rate=drop_rate)
                
            with tf.name_scope("layer2"+("_BN" if l2_batch_normal else "")):
                wL2 = tf.Variable(tf.truncated_normal([400,150]))
                bL2 = tf.Variable(tf.zeros([150]))
                mL2 = tf.matmul(rL1,wL2) + bL2
                if l2_batch_normal:
                    batch_normal2,pop_mean,pop_var = self._batch_normal(mL2,isTrain,epsilon)
                    _summary_merge_list.append(tf.summary.histogram("Mean",pop_mean))
                    _summary_merge_list.append(tf.summary.histogram("Var",pop_var))
                    rL2 = tf.nn.sigmoid(batch_normal2) 
                else:
                    rL2 = tf.nn.sigmoid(mL2) 
                rL2 = tf.nn.dropout(rL2,rate=drop_rate)
            with tf.name_scope("output"):
                wL3 = tf.Variable(tf.truncated_normal([150,10]))
                bL3 = tf.Variable(tf.zeros([10]))
                logits = tf.add(tf.matmul(rL2,wL3),bL3,name="logits")
                
            self.prediction = tf.nn.softmax(logits,name="prediction")

            #代价函数
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(y,logits),name="loss")

            #梯度下降法
            self.train = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)

            #结果存在一个布尔型列表中。m*1
            predictionIndex = tf.argmax(self.prediction,1)
            realIndex = tf.argmax(y,1)
            correct_prediction = tf.equal(predictionIndex,realIndex)

            #update learning_rate
            self.update_rate = tf.assign(self.learning_rate,self.learning_rate*0.95,name="update_lr")

            #求准确率
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="Accuracy")
            #merge summary
            self._merge_summary(_summary_merge_list)
    def _merge_summary(self,summary_merge_list):
        summary_merge_list.append(tf.summary.scalar("Loss",self.loss))
        summary_merge_list.append(tf.summary.scalar("Accuracy",self.accuracy))
        summary_merge_list.append(tf.summary.scalar("LearningRate",self.learning_rate))
        self.summay_merge = tf.summary.merge(summary_merge_list)

In [4]:
tf.reset_default_graph()

#batch_size
batch_size = 128
#计算批次
n_batch = math.ceil(mnist.train.num_examples / batch_size)

x = tf.placeholder(tf.float32,[None,784],name="input_x") #28*28=784
y = tf.placeholder(tf.float32,[None,10],name="label_y") #28*28=784
drop_rate = tf.placeholder_with_default(0.1,None,name="drop_rate")
isTrain = tf.placeholder_with_default(False,None,name="is_train")

L2_BN_MODEL = "BN L2"

model = Model(False,True,x,y,drop_rate,isTrain,L2_BN_MODEL)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


**Debugger is waiting for Session.run() connections...**
* tf.Session:
```
   import tensorflow as tf  
   from tensorflow.python import debug as tf_debug 
   
   sess = tf.Session()  
   sess = tf_debug.TensorBoardDebugWrapperSession(sess, "127.0.0.1:6007")  
   sess.run(my_fetches)  
```
 
   
* Estimator | MonitoredSession:  
```
   import tensorflow as tf  
   from tensorflow.python import debug as tf_debug  

   hook = tf_debug.TensorBoardDebugHook("127.0.0.1:6007")  
   my_estimator.fit(x=x_data, y=y_data, steps=1000, monitors=[hook])  
```

  
* Keras Model:
```
   import tensorflow as tf  
   from tensorflow.python import debug as tf_debug  
   import keras  
  
   keras.backend.set_session(  
      tf_debug.TensorBoardDebugWrapperSession(tf.Session(), "127.0.0.1:6007"))  
   Define your keras model, called "model".  
   model.fit(...)  
```

In [ ]:
##目前windows上grpc有问题，并不能成功连接到tensorboard

from tensorflow.python import debug as tf_debug
with tf.Session() as sess:
    sess = tf_debug.TensorBoardDebugWrapperSession(sess, "127.0.0.1:6007",send_traceback_and_source_code =False,log_usage=False)
    sess.run(tf.global_variables_initializer())
    with tf.summary.FileWriter("./debugger/",graph=sess.graph,session=sess) as logWriter:
        for loop in range(50):
            for batch in range(n_batch):
                batch_xs,batch_ys = mnist.train.next_batch(batch_size)
                _ = sess.run(model.train,feed_dict={x:batch_xs,y:batch_ys,drop_rate:0.1,isTrain:True})
                #logWriter.add_summary(summary,loop*n_batch+batch) 
            else:
                learningRateValue = sess.run(model.update_rate)
                test_acc = sess.run(model.accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,drop_rate:0,isTrain:False})
                train_acc = sess.run(model.accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels,drop_rate:0,isTrain:False})
                if loop % 10 == 0:
                    print("loop = {}, model = {},   test_acc = {},   train_acc = {}, learning rate = {}".format(loop,model.name,test_acc,train_acc,learningRateValue))

loop = 0, model = BN L2,   test_acc = grpc:// debug URL scheme is not implemented on Windows yet.,   train_acc = grpc:// debug URL scheme is not implemented on Windows yet., learning rate = grpc:// debug URL scheme is not implemented on Windows yet.
loop = 10, model = BN L2,   test_acc = grpc:// debug URL scheme is not implemented on Windows yet.,   train_acc = grpc:// debug URL scheme is not implemented on Windows yet., learning rate = grpc:// debug URL scheme is not implemented on Windows yet.
loop = 20, model = BN L2,   test_acc = grpc:// debug URL scheme is not implemented on Windows yet.,   train_acc = grpc:// debug URL scheme is not implemented on Windows yet., learning rate = grpc:// debug URL scheme is not implemented on Windows yet.
